In [13]:
%cd LLaVA

[Errno 2] No such file or directory: 'LLaVA'
/home/jupyter/opthollm/LLaVA


In [14]:
import argparse
import torch

from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria

from PIL import Image

import requests
from PIL import Image
from io import BytesIO
from transformers import TextStreamer

### Define Helper Methods

In [15]:
def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image



# get random file methods

In [16]:
model_path = "liuhaotian/LLaVA-Lightning-MPT-7B-preview"
model_base = None
image_file = "https://llava-vl.github.io/static/images/view.jpg"
num_gpus = 1
args_conv_mode = None
max_new_tokens = 512
temperature = 0.2
debug = False

### Initalize the Model

In [17]:
disable_torch_init()

model_name = get_model_name_from_path(model_path)
tokenizer, model, image_processor, context_len = load_pretrained_model(model_path, model_base, model_name)

if "v1" in model_name.lower():
    conv_mode = "llava_v1"
elif "mpt" in model_name.lower():
    conv_mode = "mpt"
else:
    conv_mode = "llava_v0"

if args_conv_mode is not None and conv_mode != args_conv_mode:
    print('[WARNING] the auto inferred conversation mode is {}, while `--conv-mode` is {}, using {}'.format(conv_mode, args_conv_mode, args_conv_mode))
else:
    args_conv_mode = conv_mode

conv = conv_templates[args_conv_mode].copy()
if "mpt" in model_name.lower():
    roles = ('user', 'assistant')
else:
    roles = conv.roles

You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.11.self_attn.q_proj.bias', 'text_model.encoder.layers.9.self_attn.v_proj.weight', 'text_model.encoder.layers.3.self_attn.k_proj.weight', 'text_model.encoder.layers.7.self_attn.k_proj.weight', 'text_model.encoder.layers.4.mlp.fc2.weight', 'text_model.encoder.layers.3.mlp.fc2.bias', 'text_model.encoder.layers.2.self_attn.v_proj.bias', 'text_model.encoder.layers.9.self_attn.k_proj.weight', 'text_model.encoder.layers.2.layer_norm1.bias', 'text_model.encoder.layers.0.self_attn.v_proj.bias', 'text_model.encoder.layers.11.layer_norm2.weight', 'text_model.final_layer_norm.weight', 'text_model.embeddings.position_ids', 'text_model.encoder.layers.5.layer_norm2.weight', 'text_model.encoder.layers.9.mlp.fc2.weight', 'text_model.encoder.layers.9.self_attn.out_proj.bias', 'text_model.encoder.layers.0.self_attn.k_proj.weight', 'text_model.encoder.layers.7.

In [18]:
image = load_image(image_file)
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()

In [ ]:
# try to get it to just analyze one image and then accept another



In [ ]:
while True:
        try:
            inp = input(f"{roles[0]}: ")
        except EOFError:
            inp = ""
        if not inp:
            print("exit...")
            break

        print(f"{roles[1]}: ", end="")

        if image is not None:
            # first message
            if model.config.mm_use_im_start_end:
                inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
            else:
                inp = DEFAULT_IMAGE_TOKEN + '\n' + inp
            conv.append_message(conv.roles[0], inp)
            image = None
        else:
            # later messages
            conv.append_message(conv.roles[0], inp)
        conv.append_message(conv.roles[1], None)
        prompt = conv.get_prompt()

        input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        with torch.inference_mode():
            output_ids = model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=0.2,
                max_new_tokens=1024,
                streamer=streamer,
                use_cache=True,
                stopping_criteria=[stopping_criteria])

        outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()
        conv.messages[-1][-1] = outputs

        if debug:
            print("\n", {"prompt": prompt, "outputs": outputs}, "\n")


user:  Describe the image


assistant: The image features a dock on a lake, with a wooden plank extending into the water. The dock is surrounded by a beautiful, clear lake with a mountain in the background. The scene is set during the day, with the sun shining, creating a serene and picturesque atmosphere.


user:  What shouldn't I do there?


assistant: While the dock and the surrounding area are visually appealing, there are a few things you should avoid doing there. First, you should not swim in the lake, as the water might be too cold or have strong currents. Second, you should not venture too far from the dock, as the water might be too deep or have hidden obstacles that could pose a risk. Lastly, it is essential to be mindful of the surrounding environment, including the mountain and the trees, to avoid any accidents or damage to the natural landscape.


user:  thanks!


assistant: You're welcome! The image is indeed very beautiful and serene, and it's important to be aware of the surroundings and potential risks to ensure a safe and enjoyable experience.


user:  poop?


assistant: No, there is no poop in the image. The image features a dock on a lake, with a mountain in the background, and it is set during the day with the sun shining.


user:  oh okay


assistant: Yes, the image is a beautiful scene with a dock on a lake, a mountain in 